In [5]:
import pandas as pd
df=pd.read_csv("1fps_description_10k.csv")
df

,image_path,description
0,10k_frames\1039117223_frame_0.jpg,Miracle of nature - flowering oleanders in the...
1,10k_frames\1039117223_frame_1.jpg,Miracle of nature - flowering oleanders in the...
2,10k_frames\1039117223_frame_2.jpg,Miracle of nature - flowering oleanders in the...
3,10k_frames\1039117223_frame_3.jpg,Miracle of nature - flowering oleanders in the...
4,10k_frames\1039117223_frame_4.jpg,Miracle of nature - flowering oleanders in the...
...,...,...
583385,10k_frames\7218109_frame_29.jpg,Fashion show shoes
583386,10k_frames\7218109_frame_30.jpg,Fashion show shoes
583387,10k_frames\7218109_frame_31.jpg,Fashion show shoes
583388,10k_frames\7218109_frame_32.jpg,Fashion show shoes


In [6]:
import pandas as pd
import os
from tqdm import tqdm

# Load your dataframe
# Replace 'your_dataframe.csv' with the path to your CSV file
# df = pd.read_csv('your_dataframe.csv')

# Function to check if an image exists
def check_image_exists(image_path):
    return os.path.exists(image_path)

# List to hold indices of rows with missing images
missing_images = []

# Check if images exist and display progress
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    if not check_image_exists(row['image_path']):
        missing_images.append(index)

# Drop rows with missing images
df_cleaned = df.drop(missing_images)

# Save the cleaned dataframe to a new CSV file (optional)
df_cleaned.to_csv('cleaned_dataframe.csv', index=False)

print(f"Checked {df.shape[0]} images. {len(missing_images)} images were missing and those rows have been removed.")

100%|██████████| 583390/583390 [00:54<00:00, 10611.72it/s]


Checked 583390 images. 61701 images were missing and those rows have been removed.


In [7]:
df_cleaned

,image_path,description
0,10k_frames\1039117223_frame_0.jpg,Miracle of nature - flowering oleanders in the...
1,10k_frames\1039117223_frame_1.jpg,Miracle of nature - flowering oleanders in the...
2,10k_frames\1039117223_frame_2.jpg,Miracle of nature - flowering oleanders in the...
3,10k_frames\1039117223_frame_3.jpg,Miracle of nature - flowering oleanders in the...
4,10k_frames\1039117223_frame_4.jpg,Miracle of nature - flowering oleanders in the...
...,...,...
583385,10k_frames\7218109_frame_29.jpg,Fashion show shoes
583386,10k_frames\7218109_frame_30.jpg,Fashion show shoes
583387,10k_frames\7218109_frame_31.jpg,Fashion show shoes
583388,10k_frames\7218109_frame_32.jpg,Fashion show shoes


In [1]:
import os
import pandas as pd
from tqdm import tqdm
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image

# Load the model and processor


processor = BlipProcessor.from_pretrained('./blip-image-captioning-base')
model = BlipForConditionalGeneration.from_pretrained('./blip-image-captioning-base').to("cuda")

# Load the dataset
csv_path = 'cleaned_dataframe.csv'
data = pd.read_csv(csv_path)

# Generate descriptions for each frame
def generate_descriptions(image_path):
    image = Image.open(image_path)
    inputs = processor(image, return_tensors="pt").to("cuda")
    out = model.generate(**inputs)
    description = processor.decode(out[0], skip_special_tokens=True)
    return description

# Apply tqdm to the description generation
tqdm.pandas(desc="Generating descriptions")
data['generated_description'] = data['image_path'].progress_apply(generate_descriptions)

# Save the new dataset with generated descriptions
data.to_csv('10k_generated_description.csv', index=False)

Generating descriptions:   0%|          | 0/521689 [00:00<?, ?it/s]C:\Users\jaivi\anaconda3\envs\tf-gpunew\lib\site-packages\transformers\generation\utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Generating descriptions: 100%|██████████| 521689/521689 [33:52:02<00:00,  4.28it/s]   


In [2]:
data.head()

,image_path,description,generated_description
0,10k_frames\1039117223_frame_0.jpg,Miracle of nature - flowering oleanders in the...,pink flowers on olive tree
1,10k_frames\1039117223_frame_1.jpg,Miracle of nature - flowering oleanders in the...,pink flowers on olive tree
2,10k_frames\1039117223_frame_2.jpg,Miracle of nature - flowering oleanders in the...,pink flowers on olive tree
3,10k_frames\1039117223_frame_3.jpg,Miracle of nature - flowering oleanders in the...,pink flowers on a tree branch
4,10k_frames\1039117223_frame_4.jpg,Miracle of nature - flowering oleanders in the...,pink flowers on olive tree


In [3]:
original_data

NameError: name 'original_data' is not defined

In [4]:
# Load the dataset with generated descriptions
data = pd.read_csv('10k_generated_description.csv')

# Concatenate descriptions for each video
data['video_id'] = data['image_path'].apply(lambda x: x.split('_frame_')[0].split('\\')[1])

# Convert video_id to string
data['video_id'] = data['video_id'].astype(str)

# Group descriptions by video
video_descriptions = data.groupby('video_id')['generated_description'].apply(lambda x: ' '.join(x)).reset_index()

# Load the original dataset to get the final descriptions
original_data = pd.read_csv('videos_metadata.csv')

# Convert video_id to string in the original data
original_data['video_id'] = original_data['video_id'].astype(str)

# Merge to get the final training data
training_data = pd.merge(video_descriptions, original_data[['video_id', 'description']], on='video_id')

# Save the training data
training_data.to_csv('video_training_data_10k.csv', index=False)

In [5]:
training_data

,video_id,generated_description,description
0,10007216,a soldier is sitting in the airplane and looki...,Male soldier flying on a commercial plane
1,10010666,a man in a black shirt is holding a money a ma...,Man throws hundred dollar bills at the camera ...
2,1001125,two scientists working in a laboratory two sci...,Science team working with microscopes in a lab...
3,10020833,a blackbird perched on a wooden fence a blackb...,4k uhd - zone-tailed hawk (buteo albonotatus) ...
4,1002472,a man is working on a power pole a man is work...,Electricians in a bucket fixing high voltage e...
...,...,...,...
8916,9966038,two men are working on a construction site two...,"Hai duong, vietnam, april, 12; asian man using..."
8917,9969206,a woman is standing in a gym with a ball a wom...,Full length video of yoga practitioner warming...
8918,9981878,a sunset view from the roof of a building in a...,"Jannatul mualla - also known as al-hajun, is a..."
8919,9989225,a black background with a white border a black...,In an operation room a surgeon and a nurse mak...


Test Data

In [1]:
import pandas as pd
df=pd.read_csv("test_frames.csv")
df

,image_path,description
0,test_frames\1023661_frame_0.jpg,Young chinese call center businesswoman at her...
1,test_frames\1023661_frame_1.jpg,Young chinese call center businesswoman at her...
2,test_frames\1023661_frame_2.jpg,Young chinese call center businesswoman at her...
3,test_frames\1023661_frame_3.jpg,Young chinese call center businesswoman at her...
4,test_frames\1023661_frame_4.jpg,Young chinese call center businesswoman at her...
...,...,...
5617,test_frames\1057303213_frame_58.jpg,Sunny day. students three women together. hold...
5618,test_frames\1057303213_frame_59.jpg,Sunny day. students three women together. hold...
5619,test_frames\1057303213_frame_60.jpg,Sunny day. students three women together. hold...
5620,test_frames\1057303213_frame_61.jpg,Sunny day. students three women together. hold...


In [2]:
import pandas as pd
import os
from tqdm import tqdm

# Load your dataframe
# Replace 'your_dataframe.csv' with the path to your CSV file
# df = pd.read_csv('your_dataframe.csv')

# Function to check if an image exists
def check_image_exists(image_path):
    return os.path.exists(image_path)

# List to hold indices of rows with missing images
missing_images = []

# Check if images exist and display progress
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    if not check_image_exists(row['image_path']):
        missing_images.append(index)

# Drop rows with missing images
df_cleaned = df.drop(missing_images)

# Save the cleaned dataframe to a new CSV file (optional)
df_cleaned.to_csv('cleaned_test_dataframe.csv', index=False)

print(f"Checked {df.shape[0]} images. {len(missing_images)} images were missing and those rows have been removed.")

100%|██████████| 5622/5622 [00:00<00:00, 12752.35it/s]

Checked 5622 images. 0 images were missing and those rows have been removed.


In [3]:
import os
import pandas as pd
from tqdm import tqdm
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image

# Load the model and processor


processor = BlipProcessor.from_pretrained('./blip-image-captioning-base')
model = BlipForConditionalGeneration.from_pretrained('./blip-image-captioning-base').to("cuda")

# Load the dataset
csv_path = 'cleaned_test_dataframe.csv'
data = pd.read_csv(csv_path)

# Generate descriptions for each frame
def generate_descriptions(image_path):
    image = Image.open(image_path)
    inputs = processor(image, return_tensors="pt").to("cuda")
    out = model.generate(**inputs)
    description = processor.decode(out[0], skip_special_tokens=True)
    return description

# Apply tqdm to the description generation
tqdm.pandas(desc="Generating descriptions")
data['generated_description'] = data['image_path'].progress_apply(generate_descriptions)

# Save the new dataset with generated descriptions
data.to_csv('test_generated_description.csv', index=False)

Generating descriptions:   0%|          | 0/5622 [00:00<?, ?it/s]C:\Users\jaivi\anaconda3\envs\tf-gpunew\lib\site-packages\transformers\generation\utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Generating descriptions: 100%|██████████| 5622/5622 [28:38<00:00,  3.27it/s]


In [4]:
# Load the dataset with generated descriptions
data = pd.read_csv('test_generated_description.csv')

# Concatenate descriptions for each video
data['video_id'] = data['image_path'].apply(lambda x: x.split('_frame_')[0].split('\\')[1])

# Convert video_id to string
data['video_id'] = data['video_id'].astype(str)

# Group descriptions by video
video_descriptions = data.groupby('video_id')['generated_description'].apply(lambda x: ' '.join(x)).reset_index()

# Load the original dataset to get the final descriptions
original_data = pd.read_csv('test_videos.csv')

# Convert video_id to string in the original data
original_data['video_id'] = original_data['video_id'].astype(str)

# Merge to get the final training data
training_data = pd.merge(video_descriptions, original_data[['video_id', 'description']], on='video_id')

# Save the training data
training_data.to_csv('test_videodata_description.csv', index=False)

In [10]:
training_data

,video_id,generated_description,description
0,1006792915,a martini glass with a black background a mart...,Process of serving margarita cocktail. bartend...
1,1007017096,a man preparing a meal in the kitchen a woman ...,Dolly shot of cook at the kitchen cutting mush...
2,1007829319,a little boy in a red jacket and hat is feedin...,Small boy feeding pigeons on the street
3,1008414787,an aerial view of the island and the ocean a s...,Aerial view beachfront destination usa picnic ...
4,1010428679,"aerial drone video of the sunset in kotorp, ko...","Aerial view dubrovnik old town in dalmatia, cr..."
...,...,...,...
95,8096140,a moose in the woods a moose in the woods eati...,Moose in autumn - sweden - rutting season
96,8800768,a city street with cars and people walking a c...,Hong kong - circa october 2014: day light city...
97,9138830,a black and white photo of a city a black and ...,Circa 1960s - the japanese navy is welcomed to...
98,9256856,a group of people in red costumes are walking ...,"San pablo city, laguna, philippines - january ..."
